In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures


import numpy as np
import pandas as pd 


import os
for dirname, _, filenames in os.walk('/kaggle/input/user-car-prices-barcelona-2022'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/user-car-prices-barcelona-2022/used_cars_data.csv


In [3]:
df = pd.read_csv('/kaggle/input/user-car-prices-barcelona-2022/used_cars_data.csv')
df.head()

,Unnamed: 0,brand,model,price (eur),engine,year,mileage (kms),fuel,gearbox,location
0,0,SEAT,Ibiza,8990,SC 1.2 TSI 90cv Style,2016,67000,Gasolina,Manual,Granollers
1,1,Hyundai,i30,9990,1.6 CRDi 110cv Tecno,2014,104868,Diésel,Manual,Viladecans
2,2,BMW,Serie 5,13490,530d Touring,2011,137566,Diésel,Automatica,Viladecans
3,3,Volkswagen,Golf,24990,GTI 2.0 TSI 169kW (230CV),2018,44495,Gasolina,Manual,Viladecans
4,4,Opel,Corsa,10460,1.4 Expression 90 CV,2016,69800,Gasolina,Manual,Sabadell 1


In [4]:
df.isna().sum()
df.duplicated().sum()

0

Checking for null values and duplicates. No null and duplicates in this dataset

In [ ]:
for i in df.columns:
    print(df[i].value_counts())

Checking the values of different features

In [5]:
df = df.iloc[: ,1:10]

Slicing the df excluding the first column.

In [6]:
scaler = StandardScaler()
mileage_reshaped = np.array(df['mileage (kms)']).reshape(-1,1)
df['mileage_scaled'] = scaler.fit_transform(mileage_reshaped)


Scaling the mileage feature

In [7]:
df = pd.get_dummies(df)

One hot encoding using get dummies

In [ ]:
m1 = IsolationForest()
m1.fit(df)
df['anamoly']= m1.predict(df)
df.anamoly.value_counts()

No outlier detected

In [8]:
y = df['price (eur)']
y = pd.DataFrame(y)
df.drop(columns = ['price (eur)', 'mileage (kms)'], inplace = True)

In [9]:
trainx, testx, trainy, testy =  train_test_split(df, y, test_size = 0.35, random_state = 100)

Splitting into test and train

In [12]:
poly = PolynomialFeatures(degree = 2, include_bias = False, interaction_only = False)

Adding  2 polyniomial features

In [13]:
trainxp = poly.fit_transform(trainx)
columns = poly.get_feature_names_out(input_features = trainx.columns)
trainxp = pd.DataFrame(trainxp, columns= columns)
trainxp.columns = trainxp.columns.str.replace(' ', '_')
trainxp.columns = trainxp.columns.str.replace('^', '_')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """


In [14]:
testxp= poly.transform(testx)
columns = poly.get_feature_names_out(input_features = testx.columns)
testxp = pd.DataFrame(testxp, columns= columns)
testxp.columns = testxp.columns.str.replace(' ', '_')
testxp.columns = testxp.columns.str.replace('^', '_')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """


In [ ]:
model = Ridge()
alpha = np.logspace(-6, 1, 10)
max_iter = np.array(range(100000, 10000000))
solver = ['svd', 'cholesky', 'lsqr','lbfgs']
param = {'alpha': alpha, 'max_iter': max_iter, 'solver': solver}
rridge =  RandomizedSearchCV(model, param, cv = 10, scoring = 'neg_mean_squared_error')
rridge.fit(trainx, trainy)
print('alpha', rridge.best_estimator_.alpha)
print('max_iter', rridge.best_estimator_.max_iter)
print('solver', rridge.best_estimator_.solver)

Used RandomizedsearchCV for parameter tuning

In [15]:
model = Ridge(alpha = 1.66, max_iter = 8248934, solver = 'svd', random_state = 100)
model.fit(trainx, trainy)
predy = model.predict(testx)
np.sqrt(mean_squared_error(testy, predy, squared = False))


65.73871261888961

# The best model


RMSE of 65.7

In [ ]:
model = Lasso()
alpha = np.linspace(0, 1, 100)
max_iter = np.array(range(100000, 10000000))
param = {"alpha": alpha, 'max_iter': max_iter}
rlasso = RandomizedSearchCV(model, param, cv= 5, scoring = 'neg_mean_squared_error')
rlasso.fit(trainx,trainy)
print('alpha', rlasso.best_estimator_.alpha)
print('max_iter', rlasso.best_estimator_.max_iter)


Used RandomizedsearchCV for parameter tuning

In [16]:
model = Lasso(alpha =0.888888888888889, max_iter = 1217846, random_state = 100 )
model.fit(trainx, trainy)
predy = model.predict(testx)
np.sqrt(mean_squared_error(testy, predy, squared = False))


83.30490350213526

RMSE of 83.3

In [ ]:
model = ElasticNet()
alpha = np.linspace(0, 1, 100)
max_iter = np.array(range(100000, 10000000))
param = {'alpha': alpha, 'max_iter': max_iter}
relastic = RandomizedSearchCV(model, param, cv = 10, scoring = 'neg_mean_squared_error' )
relastic.fit(trainx, trainy)
print(relastic.best_params_)


Used RandomizedsearchCV for parameter tuning

In [17]:
model = ElasticNet(alpha =0.010101010101010102, max_iter = 9942387 , random_state = 100)
model.fit(trainx, trainy)
predy = model.predict(testx)
np.sqrt(mean_squared_error(testy, predy, squared = False))

66.46744423491855

RMSE of 66.4

In [ ]:
model = DecisionTreeRegressor()
criterion = ['squared_error', 'friedman_mse', 'poisson', 'absolute_error']
min_samples_split = np.array(range(1, 100))
max_depth = np.array(range(1, 100))
param = {'criterion' : criterion, 'min_samples_split' : min_samples_split, 'max_depth': max_depth}
rdecision = RandomizedSearchCV(model, param, cv=10, scoring = 'neg_mean_squared_error' )
rdecision.fit(trainx, trainy)
print(rdecision.best_params_)

Used RandomizedsearchCV for parameter tuning

In [18]:
model = DecisionTreeRegressor(criterion = 'friedman_mse' , min_samples_split = 45 , max_depth = 34 , random_state = 100)
model.fit(trainx, trainy)
predy = model.predict(testx)
np.sqrt(mean_squared_error(testy, predy, squared = False))

81.39671528328171

RMSE of 81.3

In [ ]:
model = RandomForestRegressor()
criterion = ['squared_error', 'friedman_mse', 'poisson', 'absolute_error']
min_samples_split = np.array(range(1, 100))
max_depth = np.array(range(1, 100))
n_estimators = np.array(range(1, 100))
param = {'criterion': criterion, 'min_samples_split': min_samples_split, 'max_depth': max_depth, 'n_estimators': n_estimators}
rrandom = RandomizedSearchCV(model, param, cv=10, scoring = 'neg_mean_squared_error')
rrandom.fit(trainx, trainy.values.ravel())
print(rrandom.best_params_)

Used RandomizedsearchCV for parameter tuning

In [19]:
model = RandomForestRegressor(n_estimators = 83, min_samples_split = 8, max_depth = 52, criterion = 'absolute_error', random_state = 100)
model.fit(trainx, trainy.values.ravel())
predy = model.predict(testx)
np.sqrt(mean_squared_error(testy, predy, squared = False))

75.22538225638058

RMSE of 75.2

In [ ]:
model = AdaBoostRegressor()
n_estimators = np.array(range(1, 100))
learning_rate = np.linspace(0,1, 100)
param = {'n_estimators': n_estimators, 'learning_rate': learning_rate}
rada = RandomizedSearchCV(model, param, cv=10, scoring = 'neg_mean_squared_error')
rada.fit(trainx, trainy.values.ravel())
print(rada.best_estimator_)

Used RandomizedsearchCV for parameter tuning

In [20]:
model = AdaBoostRegressor(learning_rate = 0.08080808080808081, n_estimators = 40, random_state = 100)
model.fit(trainx, trainy.values.ravel())
predy = model.predict(testx)
np.sqrt(mean_squared_error(testy, predy, squared = False))

76.97899878229637

RMSE of 76.9

In [ ]:
model = BaggingRegressor()
n_estimators = np.array(range(1, 100))
max_features = np.array(range(1, 955))
param = {'n_estimators': n_estimators, 'max_features': max_features}
rbag = RandomizedSearchCV(model, param, cv=10, scoring = 'neg_mean_squared_error')
rbag.fit(trainx, trainy.values.ravel())
print(rbag.best_estimator_)

Used RandomizedsearchCV for parameter tuning

In [21]:
model = BaggingRegressor(max_features = 805, n_estimators = 2)
model.fit(trainx, trainy.values.ravel())
predy = model.predict(testx)
np.sqrt(mean_squared_error(testy, predy, squared = False))

74.29099769512015

RMSE of 74.2

In [ ]:
model = ExtraTreesRegressor()
criterion = ['squared_error', 'friedman_mse', 'poisson', 'absolute_error']
min_samples_split = np.array(range(1, 100))
max_depth = np.array(range(1, 100))
n_estimators = np.array(range(1, 100))
param = {'criterion': criterion, 'min_samples_split': min_samples_split, 'max_depth': max_depth, 'n_estimators': n_estimators}
rextra = RandomizedSearchCV(model, param, cv=10, scoring = 'neg_mean_squared_error')
rextra.fit(trainx, trainy.values.ravel())
print(rextra.best_params_)

Used RandomizedsearchCV for parameter tuning

In [22]:
model = ExtraTreesRegressor(n_estimators = 87, min_samples_split = 23, max_depth = 72, criterion = 'absolute_error')
model.fit(trainx, trainy.values.ravel())
predy = model.predict(testx)
np.sqrt(mean_squared_error(testy, predy, squared = False))

75.47224848746575

RMSE of 75.4